In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup

In [ ]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import emcee

import pymc3 as pm
import theano.tensor as tt

import exoplanet as xo

url = "epic203771098.csv"
data = pd.read_csv(url, index_col=0)

x = np.array(data.t)
y = np.array(data.vel)
yerr = np.array(data.errvel)

plt.errorbar(x, y, yerr=yerr, fmt=".k")
plt.xlabel("time [days]")
plt.ylabel("radial velocity [m/s]");

In [ ]:
periods = [20.8851, 42.3633]
period_errs = [0.0003, 0.0006]
t0s = [2072.7948, 2082.6251]
t0_errs = [0.0007, 0.0004]
Ks = xo.estimate_semi_amplitude(periods, x, y, yerr, t0s=t0s)
print(Ks, "m/s")

In [ ]:
with pm.Model() as model:
        
    # Gaussian priors based on transit data (from Petigura et al.)
    t0 = pm.Normal("t0", mu=np.array(t0s), sd=np.array(t0_errs), shape=2)
    BoundedNormal = pm.Bound(pm.Normal, lower=0.0, upper=100.0)
    P = BoundedNormal(
        "P", mu=np.array(periods), sd=np.array(period_errs), shape=2,
        testval=np.array(periods))

    # Wide log-normal prior for semi-amplitude
    logK = pm.Uniform("logK", lower=0, upper=np.log(50),
                      testval=np.log(Ks), shape=2)
        
    # Jitter & a quadratic RV trend
    logs = pm.Normal("logs", mu=np.log(np.median(yerr)), sd=5.0)
    trend = pm.Normal("trend", mu=0, sd=10.0**-np.arange(3)[::-1], shape=3)
        
    # Set up the orbit
    orbit = xo.orbits.KeplerianOrbit(period=P, t0=t0)
    
    # Set up the RV model and save it as a deterministic
    # for plotting purposes later
    vrad = orbit.get_radial_velocity(x, K=tt.exp(logK))
    
    # Define the background model
    A = np.vander(x - 0.5*(x.min() + x.max()), 3)
    bkg = tt.dot(A, trend)
    
    # Sum over planets and add the background to get the full model
    rv_model = tt.sum(vrad, axis=-1) + bkg
    
    # Likelihood
    err = tt.sqrt(yerr**2 + tt.exp(2*logs))
    pm.Normal("obs", mu=rv_model, sd=err, observed=y)
    
    map_soln = model.test_point
    map_soln = xo.optimize(start=map_soln, vars=[trend])
    map_soln = xo.optimize(start=map_soln, vars=[logs])
    map_soln = xo.optimize(start=map_soln, vars=[logK])
    map_soln = xo.optimize(start=map_soln)

In [ ]:
np.random.seed(42)
sampler = xo.PyMC3Sampler(start=200, window=100, finish=300)
with model:
    burnin = sampler.tune(tune=4000, start=model.test_point,
                          step_kwargs=dict(target_accept=0.95))

In [ ]:
with model:
    strt = time.time()
    trace = sampler.sample(draws=5000, cores=1)
    time_xo = time.time() - strt

In [ ]:
xo_samples = np.array(trace.get_values("logK", combine=False))
xo_samples = np.moveaxis(xo_samples, 0, 1)
tau_xo = emcee.autocorr.integrated_time(xo_samples)
neff_xo = np.prod(xo_samples.shape[:2]) / tau_xo
teff_xo = time_xo / neff_xo
teff_xo

In [ ]:
np.random.seed(42)

with model:
    func = xo.get_theano_function_for_var([model.logpt] + model.deterministics)
    
    def logprob(theta):
        point = model.bijection.rmap(theta)
        args = xo.get_args_for_theano_function(point)
        return tuple(func(*args))
    
    x0 = model.bijection.map(map_soln)
    initial_blobs = logprob(x0)[1:]
    dtype = [(var.name, float, np.shape(b)) for var, b in
             zip(model.deterministics, initial_blobs)]
    nwalkers = 36
    ndim = len(x0)
    x0 = x0 + 1e-5 * np.random.randn(nwalkers, ndim)
    
    emcee_sampler = emcee.EnsembleSampler(nwalkers, ndim, logprob, blobs_dtype=dtype)
    state = emcee_sampler.run_mcmc(x0, 2000, progress=True)
    emcee_sampler.reset()
    strt = time.time()
    emcee_sampler.run_mcmc(state, 50000, progress=True)
    time_emcee = time.time() - strt

In [ ]:
emcee_samples = emcee_sampler.get_blobs()
emcee_samples = emcee_samples["logK"]
tau_emcee = emcee.autocorr.integrated_time(emcee_samples)
neff_emcee = np.prod(emcee_samples.shape[:2]) / tau_emcee
teff_emcee = time_emcee / neff_emcee
teff_emcee

In [ ]:
def get_function(x):
    n_t, n_w = x.shape
    f = np.zeros(n_t)
    for k in range(n_w):
        f += emcee.autocorr.function_1d(x[:, k])
    f /= n_w
    return f

fig, axes = plt.subplots(2, 2, figsize=(10, 8))

ax = axes[0, 0]
t_emcee = np.linspace(0, time_emcee, emcee_samples.shape[0])
m = t_emcee < 5
ax.plot(t_emcee[m], np.exp(emcee_samples[m, 5, 0]), lw=0.75)
ax.annotate("emcee", xy=(0, 1), xycoords="axes fraction",
            ha="left", va="top", fontsize=16,
            xytext=(5, -5), textcoords="offset points")
ax.set_ylabel("$K_b$ [m/s]")
ax.set_xlim(0, 5)
ax.set_ylim(1, 8)
ax.set_xticklabels([])

ax = axes[1, 0]
t_xo = np.linspace(0, time_xo, xo_samples.shape[0])
m = t_xo < 5
ax.plot(t_xo[m], np.exp(xo_samples[m, 0, 0]), lw=0.75)
ax.annotate("exoplanet", xy=(0, 1), xycoords="axes fraction",
            ha="left", va="top", fontsize=16,
            xytext=(5, -5), textcoords="offset points")
ax.set_ylabel("$K_b$ [m/s]")
ax.set_xlabel("walltime [sec]")
ax.set_xlim(0, 5)
ax.set_ylim(1, 8)

ax = axes[0, 1]
f_emcee = get_function(np.exp(emcee_samples[:, :, 0]))
scale = time_emcee / np.prod(emcee_samples.shape[:2])
ax.plot(scale * np.arange(len(f_emcee)), f_emcee)
ax.axhline(0, color="k", lw=0.5)
ax.axvline(scale * tau_emcee[0], color="k", lw=3, alpha=0.3)
ax.annotate("emcee", xy=(1, 1), xycoords="axes fraction",
            ha="right", va="top", fontsize=16,
            xytext=(-5, -5), textcoords="offset points")
ax.set_xlim(0, 0.03)
ax.set_xticklabels([])
ax.set_yticks([])
ax.set_ylabel("autocorrelation")

ax = axes[1, 1]
f_xo = get_function(np.exp(xo_samples[:, :, 0]))
scale = time_xo / np.prod(xo_samples.shape[:2])
ax.plot(scale * np.arange(len(f_xo)), f_xo)
ax.axhline(0, color="k", lw=0.5)
ax.axvline(scale * tau_xo[0], color="k", lw=3, alpha=0.3)
ax.annotate("exoplanet", xy=(1, 1), xycoords="axes fraction",
            ha="right", va="top", fontsize=16,
            xytext=(-5, -5), textcoords="offset points")
ax.set_xlim(0, 0.03)
ax.set_yticks([])
ax.set_xlabel("walltime lag [sec]")
ax.set_ylabel("autocorrelation")

fig.subplots_adjust(hspace=0.05, wspace=0.15)
fig.savefig("simple_rv.pdf", bbox_inches="tight");